# Make All Tables for Main Manuscript

Kendra Wyant  
October 3, 2025

In [ ]:
#| message: false
#| warning: false

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
library(kableExtra)


Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows

## Data and calculations

Table 1

In [ ]:
study_dates <- read_csv(here::here(path_shared, "study_dates_messages.csv"),
                        show_col_types = FALSE)

screen <- read_csv(file.path(path_shared, "screen.csv"), 
                   col_types = cols()) |>
  filter(subid %in% study_dates$subid) |> 
  mutate(across(dsm5_1:dsm5_11, ~ recode(., "No" = 0, "Yes" = 1))) |>  
  rowwise() |>  
  mutate(dsm5_total = sum(c(dsm5_1, dsm5_2, dsm5_3, dsm5_4, dsm5_5, dsm5_6, dsm5_7, 
                              dsm5_8, dsm5_9, dsm5_10, dsm5_11))) |>  
  ungroup()

# lapses <- read_csv(file.path(path_shared, "lapses.csv"), 
#                    col_types = cols()) |>
#   filter(exclude == FALSE) |> 
#   filter(subid %in% study_dates$subid)

# Calcs to make df for table 1 
n_total <- nrow(study_dates)

dem_age <- screen |>
  summarise(mean = as.character(round(mean(dem_1, na.rm = TRUE), 1)),
            SD = as.character(round(sd(dem_1, na.rm = TRUE), 1)),
            min = as.character(min(dem_1, na.rm = TRUE)),
            max = as.character(max(dem_1, na.rm = TRUE))) |>
  mutate(var = "Age",
         n = as.numeric(""),
         perc = as.numeric("")) |>
  select(var, n, perc, everything()) 

dem_sex <-  screen |>
  select(var = dem_2) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Sex at Birth", .before = 1)

dem_race <- screen |>
  select(var = dem_3) |>
  mutate(var = fct_relevel(factor(var,
                         c("American Indian/Alaska Native", "Asian", "Black/African American",
                           "White/Caucasian", "Other/Multiracial")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Race", .before = 1)
  
  
dem_ethnicity <- screen |>
  select(var = dem_4) |>
  mutate(var = case_when(var == "No, I am not of Hispanic, Latino, or Spanish origin" ~ "No",
                         TRUE ~ "Yes"),
         var = fct_relevel(factor(var, c("Yes", "No")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Hispanic, Latino, or Spanish origin", .before = 1)

dem_education <- screen |>
  select(var = dem_5) |>
  mutate(var = fct_relevel(factor(var,
                         c("Less than high school or GED degree", "High school or GED",
                           "Some college", "2-Year degree", "College degree", "Advanced degree")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Education", .before = 1)

dem_employment <- screen |>
  select(var = dem_6, dem_6_1) |>
  mutate(var = case_when(dem_6_1 == "Full-time" ~ "Employed full-time",
                         dem_6_1 == "Part-time" ~ "Employed part-time",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Employed full-time", "Employed part-time", "Full-time student",
                           "Homemaker", "Disabled", "Retired", "Unemployed",
                           "Temporarily laid off, sick leave, or maternity leave",
                           "Other, not otherwise specified")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Employment", .before = 1)

dem_income <- screen |>
  summarise(mean = format(round(mean(dem_7, na.rm = TRUE), 0), big.mark = ","),
            SD = format(round(sd(dem_7, na.rm = TRUE), 0), big.mark = ","),
            min =format(round(min(dem_7, na.rm = TRUE), 0), big.mark = ","),
            max = format(round(max(dem_7, na.rm = TRUE), 0), scientific = FALSE, big.mark = ",")) |>
  mutate(var = "Personal Income",
        n = as.numeric(""),
        perc = as.numeric(""),
        mean = str_c("$", as.character(mean)),
        SD = str_c("$", as.character(SD)),
        min = str_c("$", as.character(min)),
        max = as.character(max)) |>
  select(var, n, perc, everything())

dem_marital <- screen |>
  select(var = dem_8) |>
  mutate(var = case_when(var == "Never Married" ~ "Never married",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Never married", "Married", "Divorced", "Separated",
                           "Widowed")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Marital Status", .before = 1)

table_dem <- dem_age |> 
  bind_rows(dem_sex) |> 
  bind_rows(dem_race) |> 
  bind_rows(dem_ethnicity) |> 
  bind_rows(dem_education) |> 
  bind_rows(dem_employment) |> 
  bind_rows(dem_income) |> 
  bind_rows(dem_marital) |> 
  mutate(range = str_c(min, "-", max),
         perc = round(perc, 1)) |> 
  select(-c(min, max)) |> 
  rename(` ` = var,
         N = n,
         `%` = perc,
         M = mean, 
         Range = range)

### Table 2: Demographics

In [ ]:
#| label: tbl-2
#| tbl-cap: "Demographics"

footnote_table_dem_a <- "N = 146"


table_dem |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::group_rows(start_row = 3, end_row = 4) |> 
  kableExtra::group_rows(start_row = 6, end_row = 10) |> 
  kableExtra::group_rows(start_row = 12, end_row = 13) |> 
  kableExtra::group_rows(start_row = 15, end_row = 20) |> 
  kableExtra::group_rows(start_row = 22, end_row = 30) |> 
  kableExtra::group_rows(start_row = 33, end_row = 37) |> 
  kableExtra::footnote(general = c(footnote_table_dem_a), escape=FALSE)

N 
 % 
 M 
 SD 
 Range 
 
 
 
 
 Age 
 
 
 40.5 
 11.8 
 21-72 
 
 
 Sex at Birth 
 
 
 
 
 
 
 
 
 Female 
 74 
 50.7 
 
 
 
 
 
 Male 
 72 
 49.3 
 
 
 
 
 
 Race 
 
 
 
 
 
 
 
 
 American Indian/Alaska Native 
 3 
 2.1 
 
 
 
 
 
 Asian 
 2 
 1.4 
 
 
 
 
 
 Black/African American 
 8 
 5.5 
 
 
 
 
 
 White/Caucasian 
 127 
 87.0 
 
 
 
 
 
 Other/Multiracial 
 6 
 4.1 
 
 
 
 
 
 Hispanic, Latino, or Spanish origin 
 
 
 
 
 
 
 
 
 Yes 
 3 
 2.1 
 
 
 
 
 
 No 
 143 
 97.9 
 
 
 
 
 
 Education 
 
 
 
 
 
 
 
 
 Less than high school or GED degree 
 1 
 0.7 
 
 
 
 
 
 High school or GED 
 14 
 9.6 
 
 
 
 
 
 Some college 
 40 
 27.4 
 
 
 
 
 
 2-Year degree 
 13 
 8.9 
 
 
 
 
 
 College degree 
 56 
 38.4 
 
 
 
 
 
 Advanced degree 
 22 
 15.1 
 
 
 
 
 
 Employment 
 
 
 
 
 
 
 
 
 Employed full-time 
 71 
 48.6 
 
 
 
 
 
 Employed part-time 
 25 
 17.1 
 
 
 
 
 
 Full-time student 
 7 
 4.8 
 
 
 
 
 
 Homemaker 
 1 
 0.7 
 
 
 
 
 
 Disabled 
 7 
 4.8 
 
 
 
 
 
 Retired 
 8 
 5.5 
 
 
 
 
 
 Unemployed 
 15 
 10.3 
 
 
 
 
 
 Temporarily laid off, sick leave, or maternity leave 
 3 
 2.1 
 
 
 
 
 
 Other, not otherwise specified 
 9 
 6.2 
 
 
 
 
 
 Personal Income 
 
 
 $34,726 
 $31,978 
 $0-200,000 
 
 
 Marital Status 
 
 
 
 
 
 
 
 
 Never married 
 64 
 43.8 
 
 
 
 
 
 Married 
 32 
 21.9 
 
 
 
 
 
 Divorced 
 43 
 29.5 
 
 
 
 
 
 Separated 
 5 
 3.4 
 
 
 
 
 
 Widowed 
 2 
 1.4 
 
 
 
 
 
 
 Note: 
 
 N = 146